## Init Connection

In [1]:
%load_ext sql
%sql hive://hadoop@localhost:10000/text

## Saving the result to a new table

In [2]:
%%sql
CREATE TABLE word_gutenberg
AS select lower(word) as word from (
    select explode(sentence) word from (
    select explode(sentences(trim(line))) sentence from raw_gutenberg where line != ''
    ) sentence_table 
) word_table

 * hive://hadoop@localhost:10000/text
Done.


[]

# Word Count

In [3]:
%%sql 
CREATE TABLE word_count_gutenberg 
AS
    SELECT
        word, count(word) as count
    FROM
        word_gutenberg
    GROUP BY
        word
    ORDER BY
        count DESC

 * hive://hadoop@localhost:10000/text
Done.


[]

In [8]:
%sql select * from word_count_gutenberg where word in ('he', 'she', 'it')

 * hive://hadoop@localhost:10000/text
Done.


word,count
it,13486110
he,13260163
she,4748052


In [10]:
%sql select * from word_count_gutenberg limit 10

 * hive://hadoop@localhost:10000/text
Done.


word,count
the,94796221
of,53179079
and,44288333
to,36378081
a,32264699
in,28327253
that,15478199
i,13664329
was,13633950
it,13486110


# Comparing Gutenberg WordCount with OEC Rank for the Top 20 Words

From Wikipedia [100 most common words](https://en.wikipedia.org/wiki/Most_common_words_in_English)

Can you compare our findings with the ones listed here (from wikipedia)

|word|place|
| ----------- | ----------- |
|the|1|
|be|2|
|to|3|
|of|4|
|and|5|
|a|6|
|in|7|
|that|9|
|have|9|
|i|10|
|it|11|
|for|12|
|not| 13|
|on|14|
|with|15|
|he|16|
|as| 17|
|you|18|
|do|19|
|at|20|

In [24]:
%%sql gutenberg_top_20 <<
SELECT *, ROW_NUMBER() OVER () AS gutenberg_place FROM (
    SELECT word FROM word_count_gutenberg LIMIT 20
) ranked_words

 * hive://hadoop@localhost:10000/text
Done.
Returning data to local variable gutenberg_top_20


In [25]:
%%sql oec_top_20 <<
select explode(map
               ('the',1,'be',2,'to',3,'of',4,'and',5,'a',6,'in',7,'that',9,'have',9,'i',10,'it',11,'for',12,'not', 13,'on',14,'with',15,'he',16,'as', 17,'you',18,'do',19,'at',20)
              ) as (word,oec_place)

 * hive://hadoop@localhost:10000/text
Done.
Returning data to local variable oec_top_20


In [26]:
df_gutenberg = gutenberg_top_20.DataFrame()
df_oec = oec_top_20.DataFrame()

In [30]:
df_gutenberg.merge(
    right = df_oec,
    how="outer",
)

,word,gutenberg_place,oec_place
0,the,1.0,1.0
1,of,2.0,4.0
2,and,3.0,5.0
3,to,4.0,3.0
4,a,5.0,6.0
5,in,6.0,7.0
6,that,7.0,9.0
7,i,8.0,10.0
8,was,9.0,NaN
9,it,10.0,11.0
